In [1]:
#from veneer.manage import create_command_line

#import veneer-py

from veneer.manage import start, create_command_line, kill_all_now
import veneer
import geopandas
import pandas as pd

In [2]:
veneer_install = 'D:/PythonScriptingForGBR/VeneerPlugin/v411'
source_version = '4.1.1'
cmd_directory = 'C:\\tempveneer\\veneer_cmd'
path = create_command_line(veneer_install,source_version,dest=cmd_directory)
path

'C:\\tempveneer\\veneer_cmd\\FlowMatters.Source.VeneerCmd.exe'

In [3]:
catchment_project='D:\\Xike\\bu\\bu_reefhydro_bo_411_noMP.rsproj'
num_copies=1    # Important - set this to be a number ~ the number of CPU cores in your system!
first_port=9950

In [4]:
processes,ports = start(catchment_project,
                        n_instances=num_copies,
                        ports=first_port,
                        debug=True,
                        veneer_exe=path,
                        remote=False)

Starting C:\tempveneer\veneer_cmd\FlowMatters.Source.VeneerCmd.exe -p 9950 -s  "D:\Xike\bu\bu_reefhydro_bo_411_noMP.rsproj"
[0] Loading plugins

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\DERMTools.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\PestFileProductionTools.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\RebuildInternalSubcatchmentRaster.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\SneakyLinkModelEditor.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\ReefHydroCalModels.dll

[0] Plugins loaded (5/5)

[0] Opening project file: D:\Xike\bu\bu_reefhydro_bo_411_noMP.rsproj

ERROR[0] log4net:ERROR Failed to find configuration section 'log4net' in the application's .config file. Check your .config file for the <log4net> and <configSections> elements. The configuration section should look like: <section name="log4net" type="log4net.Config.Log4NetConfigurationSectionHandler,log4net" />

[0] L

In [5]:
vs = [veneer.Veneer(port=p) for p in ports]

In [6]:
#kill_all_now(processes)

In [7]:
%matplotlib inline
v = vs[0]
#v.network().as_dataframe().plot()

In [8]:
sDate = v.model.get('scenario.CurrentConfiguration.StartDate')
print(sDate)

eDate = v.model.get('scenario.CurrentConfiguration.EndDate')
print(eDate)

#v.run_model()

1/01/2000 12:00:00 AM
31/03/2000 11:00:00 PM


In [9]:
the_network = v.network()
outlets = the_network.outlet_nodes()
outlets

[{'geometry': {'coordinates': [1589773.63051, -2283475.4690071], 'type': 'Point'}, 'type': 'Feature', 'properties': {'icon': '/resources/ConfluenceNodeModel', 'feature_type': 'node', 'schematic_location': [0, 0], 'name': 'Node on catchment SC #113'}, 'id': '/network/nodes/258'}]

In [34]:
def processLinks(theNetwork, allLinkAreas, currentLink, knownDownStreamLinksToUpdate):
    
    theLinkName = currentLink['properties']['name']
    theLinkID = currentLink['id']
    
    # I think this will actually return a catchment record, not just an ID
    theCat = theNetwork['features'].find_by_link(theLinkID)[0]
    
    #print(theCat)
    
    thisCatName = theCat['properties']['name']
    thisCatArea = theCat['properties']['areaInSquareMeters']
    
    #print("Processing " + thisCatName + " with area of " + str(thisCatArea))
    
    allLinkAreas[theLinkName] += thisCatArea
    
    newDSLinksToUpdate = []
    for eachDSLink in knownDownStreamLinksToUpdate:
        lName = str(eachDSLink)
        #print("DS Link " + lName + " with area of ")
        allLinkAreas[lName] += thisCatArea
        newDSLinksToUpdate.append(lName)
    
    # now add the link we've just processed
    newDSLinksToUpdate.append(theLinkName)
    
    #print("Link info: " + str(currentLink))
    
    fNodeID = currentLink['properties']['from_node']
    
    #print("Node info: " + fNodeID)
    
    fNode = the_network['features'].find_by_id(fNodeID)
    #print(fNode)
    
    fNodeName = fNode[0]['properties']['name']
    #print(fNodeName)
    
    ## recurse
    #print(the_network.upstream_links(fNode[0]))
    for upLink in the_network.upstream_links(fNode[0]):
        #tNodeID = upLink['properties']['to_node']
        #tNode = the_network['features'].find_by_id(tNodeID)
        #print("Now processing: " + upLink['properties']['name'] + " upstream of " + tNode[0]['properties']['name'] + " with initial Region: " + currentReg)
        processLinks(theNetwork, allLinkAreas, upLink, newDSLinksToUpdate)

    return

In [35]:
noRegStr = 'NoReg'
#gaugeList = ['120219A','120206A','link for catchment SC #139']
gaugeList = ['120205A','120206A','120207A','120209A','120212A','120213A','120214A','120219A']

# will assign regions to links, pair with subcatchs late
all_links = the_network['features'].find_by_feature_type('link')
linkAreas = {}

## Populate the dictionary of relationships first
for link in all_links:
    linkAreas[link['properties']['name']] = 0

## now process the network
for outNode in outlets:
    curReg = noRegStr
    
    #print(outNode.keys())
    #print(outNode['properties'].keys())
    #print(outNode['properties']['name'])
    
    thisNodeName = outNode['properties']['name']
    
    #this will be an empty list, built up for each link as we get to it
    downStreamLinksToUpdate = []
    
    ## now we begin processing
    for upLink in the_network.upstream_links(outNode):
        processLinks(the_network, linkAreas, upLink, downStreamLinksToUpdate)
    

In [36]:
linkAreas_df = pd.DataFrame(list(linkAreas.items()), columns=['LinkName','AreaM2'])
linkAreas_df
#help(v.network)
#help(run_results2)

,LinkName,AreaM2
0,link for catchment SC #155,1.810223e+09
1,link for catchment SC #211,5.786460e+07
2,link for catchment SC #158,1.120689e+08
3,link for catchment SC #147,2.194893e+08
4,link for catchment SC #192,1.484631e+08
5,link for catchment SC #141,1.076877e+08
6,link for catchment SC #218,1.077579e+08
7,link for catchment SC #114,9.444964e+09
8,link for catchment SC #216,6.374700e+07
9,link for catchment SC #175,8.416890e+07


In [41]:
network_df = v.network().as_dataframe()
catchdf = network_df[network_df.feature_type=='catchment']
linkdf = network_df[network_df.feature_type=='link']

catchlinkdf = pd.merge(catchdf, linkdf, how='inner', left_on=['link'], right_on=['id'])
catchlinkdf = catchlinkdf.rename(columns={'areaInSquareMeters_y':'linkarea', 'areaInSquareMeters_x':'areaInSquareMeters', 'feature_type_y':'link_feat_type', 'feature_type_x':'feature_type', 'geometry_y':'link_geom', 'geometry_x':'geometry', 'name_y': 'link_name'})

catchlinkdfregdf = pd.merge(catchlinkdf, linkAreas_df, how='inner', left_on=['link_name'], right_on=['LinkName'])
catchlinkdfregdf = catchlinkdfregdf.rename(columns={'name_x':'SubCat'})
catchlinkdfregdf.head(8)

,areaInSquareMeters,feature_type,from_node_x,geometry,icon_x,link_x,SubCat,schematic_location_x,to_node_x,id_x,...,from_node_y,link_geom,icon_y,link_y,link_name,schematic_location_y,to_node_y,id_y,LinkName,AreaM2
0,115784100.0,catchment,NaN,"(POLYGON ((1598608.63051 -2326900.4690071, 159...",NaN,/network/link/105,SC #219,NaN,NaN,/network/catchments/5397,...,/network/nodes/477,"LINESTRING (1605553.63051 -2331745.4690071, 16...",NaN,NaN,link for catchment SC #219,NaN,/network/nodes/473,/network/link/105,link for catchment SC #219,115784100.0
1,107757900.0,catchment,NaN,"(POLYGON ((1614268.63051 -2332540.4690071, 161...",NaN,/network/link/104,SC #218,NaN,NaN,/network/catchments/5748,...,/network/nodes/476,"LINESTRING (1609543.63051 -2337715.4690071, 16...",NaN,NaN,link for catchment SC #218,NaN,/network/nodes/473,/network/link/104,link for catchment SC #218,107757900.0
2,51212700.0,catchment,NaN,"(POLYGON ((1651138.63051 -2301430.4690071, 165...",NaN,/network/link/20,SC #134,NaN,NaN,/network/catchments/6927,...,/network/nodes/309,"LINESTRING (1639783.63051 -2310325.4690071, 16...",NaN,NaN,link for catchment SC #134,NaN,/network/nodes/301,/network/link/20,link for catchment SC #134,51212700.0
3,102207600.0,catchment,NaN,"(POLYGON ((1634758.63051 -2305240.4690071, 163...",NaN,/network/link/115,SC #1672,NaN,NaN,/network/catchments/7314,...,/network/nodes/295,"LINESTRING (1630783.63051 -2312785.4690071, 16...",NaN,NaN,link for catchment SC #1672,NaN,/network/nodes/293,/network/link/115,link for catchment SC #1672,549364500.0
4,117614700.0,catchment,NaN,"(POLYGON ((1653688.63051 -2307370.4690071, 165...",NaN,/network/link/21,SC #135,NaN,NaN,/network/catchments/7341,...,/network/nodes/310,"LINESTRING (1647493.63051 -2315125.4690071, 16...",NaN,NaN,link for catchment SC #135,NaN,/network/nodes/298,/network/link/21,link for catchment SC #135,117614700.0
5,55891800.0,catchment,NaN,"(POLYGON ((1649878.63051 -2366380.4690071, 164...",NaN,/network/link/85,SC #199,NaN,NaN,/network/catchments/7359,...,/network/nodes/443,"LINESTRING (1649803.63051 -2368915.4690071, 16...",NaN,NaN,link for catchment SC #199,NaN,/network/nodes/440,/network/link/85,link for catchment SC #199,55891800.0
6,143916300.0,catchment,NaN,"(POLYGON ((1687888.63051 -2378290.4690071, 168...",NaN,/network/link/116,SC #1675,NaN,NaN,/network/catchments/7629,...,/network/nodes/3383,"LINESTRING (1696243.63051 -2383915.4690071, 16...",NaN,NaN,link for catchment SC #1675,NaN,/network/nodes/391,/network/link/116,link for catchment SC #1675,143916300.0
7,57864600.0,catchment,NaN,"(POLYGON ((1624858.63051 -2341240.4690071, 162...",NaN,/network/link/97,SC #211,NaN,NaN,/network/catchments/7809,...,/network/nodes/463,"LINESTRING (1626103.63051 -2342635.4690071, 16...",NaN,NaN,link for catchment SC #211,NaN,/network/nodes/459,/network/link/97,link for catchment SC #211,57864600.0


In [42]:
catchlinkdfregdf.to_csv('D:/aa/BUContribAreas.csv', columns=['SubCat', 'LinkName', 'AreaM2'], index=False)

In [43]:
kill_all_now(processes)